# Importing modules and settings

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc

In [ ]:
from matplotlib.pyplot import rc_context

In [ ]:
import seaborn as sns

In [ ]:
import scipy as sp

General settings of Scanpy

In [ ]:
pd.options.display.float_format = "{:,.4f}".format

In [ ]:
sc.settings.verbosity = 4
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')


In [ ]:
umap_cmap = sns.light_palette('xkcd:medium blue', as_cmap = True)

# Declaring the input and output files

In [ ]:
adata = sc.read_h5ad('pristina_atlas.h5ad')

In [ ]:
adata

In [ ]:
clusteringlayer = 'leiden_1.5'

In [ ]:
with rc_context({'figure.figsize': (12, 12)}):
    sc.pl.umap(adata, color=clusteringlayer, legend_loc='on data', legend_fontoutline = 5, title= 'Clustering layer '+str(clusteringlayer), size = 30,
        frameon=False, add_outline = True)

In [ ]:
adata = adata.raw.to_adata()

In [ ]:
adata

In [ ]:
mean_exp = pd.DataFrame(columns=adata.var_names, index=adata.obs[clusteringlayer].cat.categories)                                                                                                 

for i in adata.obs[clusteringlayer].cat.categories: 
    mean_exp.loc[i] = adata[adata.obs[clusteringlayer].isin([i]),:].X.mean(0)

In [ ]:
mean_exp

In [ ]:
#nanos, #vasa, #piwi, #pcna, #mcm2

stemgenes = ['PrileiEVm016887t1', 'PrileiEVm004300t1', 'PrileiEVm003567t1', 'PrileiEVm016982t1', 'PrileiEVm003521t1']

In [ ]:
sc.pl.dotplot(adata, stemgenes, groupby=clusteringlayer, dendrogram=True, swap_axes = True, cmap = umap_cmap)

In [ ]:
mean_exp[stemgenes]

In [ ]:
ranked_exp = mean_exp[stemgenes].mean(axis = 1).sort_values(ascending = False)

In [ ]:
ranked_exp

In [ ]:
top_exp = ranked_exp[ranked_exp > 0.0500].index.to_list()

In [ ]:
top_exp

In [ ]:
ranked_counts = adata.obs[clusteringlayer].value_counts()

In [ ]:
ranked_counts

In [ ]:
bot_counts = ranked_counts[ranked_counts < 175].index.to_list()

In [ ]:
excluded = list(set(top_exp).intersection(set(bot_counts)))

In [ ]:
excluded

In [ ]:
adata = sc.read_h5ad('pristina_atlas.h5ad')

In [ ]:
adata[adata.obs[clusteringlayer].isin(excluded)]

In [ ]:
adata_excluded_removed = adata[~adata.obs[clusteringlayer].isin(excluded)].copy()

In [ ]:
adata_excluded_removed

In [ ]:
adata_excluded_removed.write('pristina_atlas_cut_PAGA.h5ad')